# Optical depth

The optical depth is defined as
$$
\tau = \int_0^{D_S} \frac{\rho(x)}{m} \, \pi u_T^2 R_E(x)^2 \, dx
$$
where we use $ x = \frac{D_L}{D_S} $, $ u_T = 1 $, and $ D_S = 8 \, \text{kpc} $.

The factor $ \frac{R_E(x)^2}{m} = \kappa D_S^2 x(1 - x) $ represents the squared Einstein radius scaled by the lens mass, where $ \kappa =\frac{4G}{c^2}$ is a constant.

The density profiles considered are as follows:

1. **Isothermal Sphere Density Profile**:
   $$
   \rho(x) = \frac{\rho_0 (R_\text{sol}^2 + R_c^2)}{R_c^2 + r(x)^2}
   $$
   where $ R_\text{sol} $ is the distance of the Sun from the Galactic center, $ R_c $ is the core radius, and $ r(x) $ gives the distance from the Galactic center to the lens at position $ x $.

2. **Navarro-Frenk-White (NFW) Density Profile**:
   $$
   \rho(x) = \frac{\rho_0}{\frac{r(x)}{R_s} \left(1 + \frac{r(x)}{R_s}\right)^2}
   $$
   where $ \rho_0 $ is a characteristic density, and $ R_s $ is the NFW scale radius.

In both cases, $ r(x) $ is given by
$$
r(x) = \sqrt{R_S^2 + (D_S x)^2 - 2 D_S R_s x \cos(l) \cos(b)}
$$
where $ l $ and $ b $ are the Galactic longitude and latitude of the source, respectively.


In [1]:
import numpy as np
import math
import astropy.units as u
import astropy.constants as c
from scipy.integrate import fixed_quad

def pos_sol(x):
    '''
    Calculates the position of the Sun relative to the Galactic center.

    Parameters:
    x (float): Fractional distance parameter, 0 ≤ x ≤ 1.

    Returns:
    r (Quantity): Distance of the Sun from the Galactic center in kiloparsecs.
    '''
    l = -32.8  # Galactic longitude in degrees
    b = 281  # Galactic latitude in degrees
    Ds = 55 * u.kpc  # Distance to the source in kiloparsecs
    angle = np.cos(math.radians(l)) * np.cos(math.radians(b))
    Rs = 8.5 * u.kpc  # Distance of the Sun from the Galactic center in kiloparsecs
    r = np.sqrt(Rs**2 + (Ds * x)**2 - 2 * Rs * (x * Ds) * angle)
    return r

def rho_NFW(r):
    '''    
    Navarro-Frenk-White (NFW) density profile for dark matter distribution.

    Parameters:
    r (Quantity): Distance from the Galactic center in kiloparsecs.

    Returns:
    density (Quantity): NFW density at distance r in units of solar masses per cubic kiloparsec.
    '''
    Rs = 21.5 * u.kpc  # Scale radius in kiloparsecs
    rho0 = 4.88e+6 * c.M_sun / u.kpc**3  # Characteristic density
    q = r / Rs
    bot = q * (1 + q)**2
    return rho0 / bot

def rho_iso(r):
    '''
    Isothermal sphere density profile.

    Parameters:
    r (Quantity): Distance from the Galactic center in kiloparsecs.

    Returns:
    density (Quantity): Density at distance r in units of solar masses per cubic kiloparsec.
    '''
    rho0 = 7.9e-3 * c.M_sun / (u.pc)**3  # Central density in solar masses per cubic parsec
    Rsol = 8.5 * u.kpc  # Solar distance to the Galactic center in kiloparsecs
    Rc = 5 * u.kpc  # Core radius in kiloparsecs
    top = rho0 * (Rsol**2 + Rc**2)
    bot = r**2 + Rc**2
    return top / bot

def Einstein_angle_sq_m(x):
    '''
    Computes the squared Einstein angle times the distance to the lens (Dl) times the lens mass (M).

    Parameters:
    x (float): Fractional distance parameter, 0 ≤ x ≤ 1.

    Returns:
    Quantity: Squared Einstein angle times Dl times M in units of meters squared kilograms.
    '''
    Ds = 55 * u.kpc  # Distance to the source in kiloparsecs
    k = 4 * c.G / c.c**2  # Constant related to the gravitational lensing effect
    return (k * x * (1 - x) * Ds**2)

def int_optical_depth(rho, x):
    '''
    Computes the integrand for optical depth calculation.

    Parameters:
    rho (function): Density function (either rho_NFW or rho_iso).
    x (float): Fractional distance parameter, 0 ≤ x ≤ 1.

    Returns:
    float: Integrand value for optical depth.
    '''
    r = pos_sol(x)
    einstein_radius_sq_m = Einstein_angle_sq_m(x)
    optical_depth_integrand = (np.pi * rho(r) * einstein_radius_sq_m)
    return optical_depth_integrand.decompose().value  # Strip units for integration

# Perform the integration over x from 0 to 1 using the fixed_quad method
optical_depth_iso, _ = fixed_quad(lambda x: int_optical_depth(rho_iso, x), 0, 1, n=10)
optical_depth_NFW, _ = fixed_quad(lambda x: int_optical_depth(rho_NFW, x), 0, 1, n=10)
print("Integrated Optical Depth (Isothermal Sphere):", optical_depth_iso)
print("Integrated Optical Depth (Navarro Frenk White):", optical_depth_NFW)


Integrated Optical Depth (Isothermal Sphere): 5.104534452369212e-07
Integrated Optical Depth (Navarro Frenk White): 3.943342232943051e-07
